In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import scipy.io.wavfile as wavfile

from util.plotting import compute_fft_plot_from_sample_rate
from util.data_io import read_rtl_raw_data, read_gqrx_raw_data
from util.filtering import low_pass_filter_complex_signal, low_pass_filter_real_signal

from util.phase_lock_loop import phase_lock_loop

## Test synchronous AM demodulation

In [ ]:
fc = 1.170E3
fs = 2.048E6

input_file = "data/gqrx_20211220_224958_126170000_2048000_fc.raw"
signal = read_gqrx_raw_data(input_file)

len(signal) / fs

In [ ]:
# AM is pretty narrow in bandwidth.
filtered_signal = low_pass_filter_complex_signal(signal, sample_rate=fs, cutoff_frequency=5E3)
filtered_signal = filtered_signal[::10]
filtered_fs = fs / 10

In [ ]:
# You can actually hear the audio with no other processing...
wavfile.write("data/test_dumb_am.wav", int(filtered_fs), np.real(filtered_signal))

In [ ]:
f,m = compute_fft_plot_from_sample_rate(filtered_signal, sampling_rate=filtered_fs)

fig = go.Figure()
fig.add_scatter(x=f[::100],y=m[::100])
fig.show()
# plt.plot(f,m)

In [ ]:
# PLL decoding only works if the signal is at an intermediate frequency. Shift up from baseband.

frequency_shift = 25E3
time_vector = np.linspace(0, len(filtered_signal) / filtered_fs, len(filtered_signal))
shift_signal = np.exp(2j*np.pi*frequency_shift*time_vector)

signal_shifted = filtered_signal * shift_signal

In [ ]:
# f,m = compute_fft_plot_from_sample_rate(signal_shifted, sampling_rate=filtered_fs)

# plt.plot(f,m)
fig = go.Figure()
fig.add_scatter(x=f[::100],y=m[::100])
fig.show()

In [ ]:
pll_output, _, _ = phase_lock_loop(signal_shifted, fs=filtered_fs, initial_frequency_estimate=frequency_shift, frequency_bandwidth=250)

In [ ]:
f,m = compute_fft_plot_from_sample_rate(pll_output, sampling_rate=filtered_fs)

# plt.plot(f,m)
fig = go.Figure()
fig.add_scatter(x=f[::100],y=m[::100])
fig.show()

In [ ]:
am_demod = np.conjugate(pll_output) * signal_shifted

am_demod = low_pass_filter_real_signal(am_demod, cutoff_frequency=5E3, sample_rate=filtered_fs)
am_demod = am_demod[::5]
audio_fs = filtered_fs / 5

In [ ]:
f,m = compute_fft_plot_from_sample_rate(am_demod, sampling_rate=filtered_fs)

plt.plot(f,m)
# fig = go.Figure()
# fig.add_scatter(x=f[::100],y=m[::100])
# fig.show()

In [ ]:
wavfile.write("data/test_am_pll.wav", int(audio_fs), am_demod)

In [ ]:
# Now, compare against the naive way of just using magnitude. To my ears, on this signal, I can't
# hear any difference in quality.
am_magnitude_demod = np.abs(filtered_signal)
am_magnitude_demod = low_pass_filter_real_signal(am_magnitude_demod, cutoff_frequency=5E3, sample_rate=filtered_fs)
am_magnitude_demod = am_magnitude_demod[::5]
audio_fs = filtered_fs / 5

wavfile.write("data/test_am_mag.wav", int(audio_fs), am_magnitude_demod)